<a href="https://colab.research.google.com/github/gabi-pacheco/HomeSwap-Customer-Churn/blob/main/Exchanges_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DATA CLEANING##

*  drop duplicates
*  drop nulls (if possible)
*  simply column names (if needed)
*  clean odd characters

##**Import libraries**

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

##**Connect to Google BigQuery**

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.cloud import bigquery

##**Link to Google BigQuery Project**

In [ ]:
project_id = 'savvy-torch-421309'
client = bigquery.Client(project=project_id)

exchanges = client.query('''
  SELECT * FROM `savvy-torch-421309.home_exchange.exchanges`''').to_dataframe()

##**Initial overview table**

In [ ]:
## exchanges.head()

,conversation_id,exchange_id,created_at,creator_id,guest_user_id,host_user_id,finalized_at,canceled_at,start_on,end_on,...,night_count,user_cancellation_id,exchange_type,home_type,residence_type,capacity,country,region,department,city
0,41632476,63756034,2021-09-26,2977886,2977886,1346925,NaT,NaT,2021-11-24,2021-11-28,...,4,<NA>,NON_RECIPROCAL,Home,primary,6,USA,Californie,Santa Barbara,Santa Barbara
1,41632730,63756339,2021-09-26,2992978,2784701,2992978,NaT,NaT,2023-02-04,2023-04-15,...,70,<NA>,RECIPROCAL,Home,primary,7,CHE,Sankt Gallen,Sarganserland,Vilters
2,46516060,69263722,2022-08-04,1235361,1235361,2999441,NaT,NaT,2022-08-19,2022-08-22,...,3,<NA>,NON_RECIPROCAL,Home,secondary,7,USA,Tennessee,Sevier County,Gatlinburg
3,42651231,64895824,2022-01-08,3021649,3021649,273822,NaT,NaT,2022-02-07,2022-02-26,...,19,<NA>,NON_RECIPROCAL,Home,primary,7,CAN,Ontario,Simcoe County,Collingwood
4,38896583,60666407,2020-11-24,3095644,3095644,2556634,NaT,NaT,2020-12-28,2021-01-02,...,5,<NA>,NON_RECIPROCAL,Home,primary,6,USA,California,Sonoma County,Bodega Bay


In [ ]:
## exchanges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11969751 entries, 0 to 11969750
Data columns (total 21 columns):
 #   Column                  Dtype              
---  ------                  -----              
 0   conversation_id         Int64              
 1   exchange_id             Int64              
 2   created_at              object             
 3   creator_id              Int64              
 4   guest_user_id           Int64              
 5   host_user_id            Int64              
 6   finalized_at            datetime64[us, UTC]
 7   canceled_at             datetime64[us, UTC]
 8   start_on                object             
 9   end_on                  object             
 10  guest_countguest_count  Int64              
 11  night_count             Int64              
 12  user_cancellation_id    Int64              
 13  exchange_type           object             
 14  home_type               object             
 15  residence_type          object             
 16

In [ ]:
exchanges.shape

(11969751, 21)

##**Drop duplicates**

In [ ]:
##not necessary; no duplicated rows in original exchanges table

##no longer run this code as it takes long and there are no duplicates in the table

##exchanges.drop_duplicates()

##**Drop nulls**

In [ ]:
exchanges.isnull().sum()

conversation_id                  0
exchange_id                      0
created_at                       0
creator_id                       0
guest_user_id                    0
host_user_id                     0
finalized_at              11571384
canceled_at               11890322
start_on                   1335625
end_on                     1346820
guest_countguest_count           0
night_count                      0
user_cancellation_id      11890284
exchange_type                    0
home_type                     1452
residence_type               44784
capacity                         3
country                       7104
region                      277096
department                  869905
city                       1175342
dtype: int64

In [ ]:
#Drop rows with home_type null of exchanges table = 0,01%

exchanges = exchanges.dropna(subset=['home_type'])

In [ ]:
#Drop rows with residence_type null = 0,37%

exchanges = exchanges.dropna(subset=['residence_type'])

In [ ]:
#Drop rows with country null as it represented 0.006% of exchanges table = 0,06%

exchanges = exchanges.dropna(subset=['country'])

In [ ]:
#Drop rows with region null of exchanges table = 2,3%

exchanges = exchanges.dropna(subset=['region'])

In [ ]:
#Drop rows with department null of exchanges table = 7,27%

exchanges = exchanges.dropna(subset=['department'])

In [ ]:
#Drop rows with city null of exchanges table = 9,8%

exchanges = exchanges.dropna(subset=['city'])

In [ ]:
#Drop rows with capacity null of exchanges table = 0,000001%

exchanges = exchanges.dropna(subset=['capacity'])

In [ ]:
#Drop rows with night_count negative of exchanges table = 0,000001%

exchanges = exchanges[exchanges['night_count']>=0]

##**Simplify column names**

In [ ]:
# Rename column 'guest_countguest_count' to 'guest_count'
exchanges = exchanges.rename(columns={'guest_countguest_count': 'guest_count'})

In [ ]:
exchanges.shape

(10062947, 21)

##**Convert objects on date types**

In [ ]:
# Convert each column to datetime
exchanges['created_at'] = pd.to_datetime(exchanges['created_at'], format='%Y-%m-%d', errors='coerce')
exchanges['start_on'] = pd.to_datetime(exchanges['start_on'], format='%Y-%m-%d', errors='coerce')
exchanges['end_on'] = pd.to_datetime(exchanges['end_on'], format='%Y-%m-%d', errors='coerce')

In [ ]:
dtypes = exchanges[['created_at', 'start_on', 'end_on']].dtypes
dtypes

created_at    datetime64[ns]
start_on      datetime64[ns]
end_on        datetime64[ns]
dtype: object

##**Add new fields with no timestamp**

In [ ]:
# Create new fields with naive datetime64[ns] (removing timezone)
exchanges['finalized_at_date'] = exchanges['finalized_at'].dt.tz_convert(None).dt.normalize()
exchanges['canceled_at_date'] = exchanges['canceled_at'].dt.tz_convert(None).dt.normalize()

##dt.tz_convert(None).dt.normalize() removes the timezone and normalizes the datetime to keep only the date part

In [ ]:
dtypes = exchanges[['finalized_at', 'canceled_at','finalized_at_date', 'canceled_at_date']].dtypes
dtypes

finalized_at         datetime64[us, UTC]
canceled_at          datetime64[us, UTC]
finalized_at_date         datetime64[us]
canceled_at_date          datetime64[us]
dtype: object

In [ ]:
exchanges

,conversation_id,exchange_id,created_at,creator_id,guest_user_id,host_user_id,finalized_at,canceled_at,start_on,end_on,...,exchange_type,home_type,residence_type,capacity,country,region,department,city,finalized_at_date,canceled_at_date
0,43687121,66088358,2022-03-06,3640066,3640066,3566772,NaT,NaT,NaT,NaT,...,NON_RECIPROCAL,Home,primary,6,USA,Californie,Santa Barbara,Santa Barbara,NaT,NaT
1,44383597,66883316,2022-04-10,592860,592860,1391251,NaT,NaT,2022-04-25,2022-04-27,...,NON_RECIPROCAL,Home,primary,10,USA,California,Santa Barbara,Santa Barbara,NaT,NaT
2,32857936,54215329,2019-04-30,2796089,2796089,1212467,NaT,NaT,2019-12-24,2019-12-30,...,NON_RECIPROCAL,Home,primary,6,USA,California,Sonoma County,Sonoma,NaT,NaT
3,35420916,56857998,2019-11-17,519652,519652,2686780,NaT,NaT,NaT,NaT,...,NON_RECIPROCAL,apartment,secondary,9,USA,Colorado,Summit County,Breckenridge,NaT,NaT
4,35618546,57062303,2019-12-03,2858139,2858139,1180072,NaT,NaT,2019-12-11,2019-12-15,...,NON_RECIPROCAL,Home,primary,6,USA,California,Alameda County,Oakland,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11969746,39720164,61600406,2021-04-03,246326,246326,2498749,NaT,NaT,NaT,NaT,...,NON_RECIPROCAL,Home,primary,8,ITA,Trentin-Haut-Adige,Bolzano,Merano,NaT,NaT
11969747,44767388,67319837,2022-04-30,2173180,1399533,2173180,NaT,NaT,2022-07-31,2022-08-07,...,RECIPROCAL,apartment,secondary,4,AUT,Vorarlberg,Bregenz,Bregenz,NaT,NaT
11969748,38440149,60162885,2020-08-23,2809016,2809016,3539130,NaT,NaT,2020-10-19,2020-10-24,...,NON_RECIPROCAL,Home,primary,6,USA,Floride,Brevard,Cape Canaveral,NaT,NaT
11969749,38608246,60348334,2020-09-19,586944,586944,3535504,NaT,NaT,2020-11-26,2020-12-04,...,NON_RECIPROCAL,Home,secondary,14,USA,Floride,Broward,Fort Lauderdale,NaT,NaT


#**Version: exchanges_1**

*   drop nulls
*   simplify col names
*   convert objects on date types
*   add fields with only date and no timestamp

In [ ]:
exchanges_1 = exchanges.copy()

In [ ]:
exchanges_1.dtypes

conversation_id                       Int64
exchange_id                           Int64
created_at                   datetime64[ns]
creator_id                            Int64
guest_user_id                         Int64
host_user_id                          Int64
finalized_at            datetime64[us, UTC]
canceled_at             datetime64[us, UTC]
start_on                     datetime64[ns]
end_on                       datetime64[ns]
guest_count                           Int64
night_count                           Int64
user_cancellation_id                  Int64
exchange_type                        object
home_type                            object
residence_type                       object
capacity                              Int64
country                              object
region                               object
department                           object
city                                 object
finalized_at_date            datetime64[us]
canceled_at_date             dat

In [ ]:
exchanges_1.shape

(10062947, 23)

##**Rows with only special characters**

*  Focus on region, department, city

In [ ]:
import pandas as pd
import re
import string

In [ ]:
# Select specific columns 'region', 'department', 'city'
filtered_columns_to_clean = exchanges_1[['region', 'department', 'city']]


In [ ]:
#find all that is not a character
## . means is any character
## .+ means one or more
## ^ means is not
## [] is the range
##r"[^a-zA-z] it includes ?? and R?iga
##^[ at the start
##]$ means the end of the strins
##]+ means 'as many as they are'

###r"^[^a-zA-z]+$

In [ ]:
pattern = "^[^a-zA-z]+$" #regex regular expression to exclude strings with characters

In [ ]:
exchanges_1['region'].str.contains(r"^[^a-zA-z]+$").sum()

715

In [ ]:
exchanges_1['department'].str.contains(r"^[^a-zA-z]+$").sum()

226

In [ ]:
exchanges_1['city'].str.contains(r"^[^a-zA-z]+$").sum()

259

In [ ]:
# Find the rows to delete where 'region' matches the pattern
to_delete_region = exchanges_1['region'].str.contains(pattern, na=False)

In [ ]:
to_delete_region.sum()

715

In [ ]:
exchanges_1 = exchanges_1[~to_delete_region]

In [ ]:
exchanges_1.shape

#original = (10062947, 23)
#new = (10062232, 23)

(10062232, 23)

In [ ]:
# Find the rows to delete where 'department' matches the pattern
to_delete_dep = exchanges_1['department'].str.contains(pattern, na=False)

In [ ]:
to_delete_dep.sum()

73

In [ ]:
exchanges_1 = exchanges_1[~to_delete_dep]

In [ ]:
exchanges_1.shape

#original = (10062947, 23)
#new_region_deleted = (10062232, 23)
#new_dep_deleted = (10062159, 23)

(10062159, 23)

In [ ]:
# Find the rows to delete where 'city' matches the pattern
to_delete_city = exchanges_1['city'].str.contains(pattern, na=False)

In [ ]:
to_delete_city.sum()

220

In [ ]:
exchanges_1 = exchanges_1[~to_delete_city]

In [ ]:
exchanges_1.shape

#original = (10062947, 23)
#new_region_deleted = (10062232, 23)
#new_dep_deleted = (10062159, 23)
#new_city_deleted = (10061939, 23)

#total deleted rows = 715 + 73 + 220 = 1,008 == 0,01%

(10061939, 23)

# **Version: exchanges_2**

* Removed rows with only special characters

In [ ]:
exchanges_2 = exchanges_1.copy()

In [ ]:
exchanges_2.shape

(10061939, 23)

##**Identify cells with special characters**

*  region, department, city

In [ ]:
# Select specific columns 'region', 'department', 'city'
filtered_columns_to_clean = exchanges_2[['region', 'department', 'city']]

In [ ]:
# Define your set of special characters (modify as needed)
special_characters_pattern = r'[!"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~]'

# Example usage:
#text = "This is a string with special characters like !@#$%^&*()_+{}[]~`"
#matches = re.findall(special_characters_pattern, text)
#print("Special characters found:", matches)

In [ ]:
# List to store results
non_alpha_regions = []

In [ ]:
# Iterate over each row in the 'region' column and find special characters
for region in exchanges_2['region']:
    matches = re.findall(special_characters_pattern, region)
    if matches:
        non_alpha_regions.extend(matches)

print("Special characters found in 'region' column:", non_alpha_regions)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Convert the list to a set to get distinct special characters
distinct_special_characters = set(non_alpha_regions)

distinct_special_characters

{"'", ',', '-', '.', '/', '?'}

🎯 **NEXT:** count entries with "?" and "/" special character

In [ ]:
# Count occurrences of specific characters '/' and '?'
count_slash_question = non_alpha_regions.count('/') + non_alpha_regions.count('?')

print(f"Total rows with '/' or '?': {count_slash_question}")

Total rows with '/' or '?': 26514


🎯**NEXT:** check entries with "/" special character

In [ ]:
# Find rows in 'region' column that contain '/'
rows_with_slash = exchanges_2[exchanges_2['region'].str.contains('/', na=False)]

# Display the rows
rows_with_slash[['region', 'department', 'city']]

,region,department,city
1367,País Vasco/Euskadi,Guipúzcoa,Guetaria
7272,Acklins/Crooked Island,Acklins/Crooked Island,Acklins/Crooked Island
16264,País Vasco/Euskadi,Guipúzcoa,San Sebastián
70074,Acklins/Crooked Island,Acklins/Crooked Island,Acklins/Crooked Island
70558,País Vasco/Euskadi,Guipúzcoa,Guetaria
...,...,...,...
11865186,País Vasco/Euskadi,Vizcaya,Alzaga
11884115,País Vasco/Euskadi,Vizcaya,Alzaga
11902819,País Vasco/Euskadi,Vizcaya,Alzaga
11912648,País Vasco/Euskadi,Albacete,Vitoria


In [ ]:
# Find rows in 'department' column that contain '/'
rows_with_slash = exchanges_2[exchanges_2['department'].str.contains('/', na=False)]

# Display the rows
rows_with_slash[['region', 'department', 'city']]

,region,department,city
2887,Graubünden,Prättigau/davos,Davos Platz
7272,Acklins/Crooked Island,Acklins/Crooked Island,Acklins/Crooked Island
24382,Fribourg,See/Lac,Morat
28238,Graubünden,Prättigau/davos,Davos Platz
41673,Graubünden,Prättigau/davos,Davos Platz
...,...,...,...
11952796,Freiburg,See/Lac,Mont-Vully
11952797,Freiburg,See/Lac,Mont-Vully
11954491,Fribourg,See/Lac,Cormondes
11966730,Freiburg,See/Lac,Mont-Vully


In [ ]:
# Find rows in 'city' column that contain '/'
rows_with_slash = exchanges_2[exchanges_2['city'].str.contains('/', na=False)]

# Display the rows
rows_with_slash[['region', 'department', 'city']]

,region,department,city
7272,Acklins/Crooked Island,Acklins/Crooked Island,Acklins/Crooked Island
17065,Sachsen,Vogtlandkreis,Auerbach/Vogtl.
31229,Graubünden,Surselva,Breil/Brigels
32847,Grisons,Imboden,Domat/Ems
41275,Graubünden,Imboden,Domat/ems
...,...,...,...
11961349,Comunidad Valenciana,Castellón,Orpesa/oropesa Del Mar
11961820,Comunidad Valenciana,Castellón,Sant Jordi/San Jorge
11966538,Comunidad Valenciana,Castellón,Orpesa/oropesa Del Mar
11967185,Grisons,Imboden,Domat/Ems


✅ leave "/", they are valid entries

**NEXT:** 🎯 target entries with "?" special character

In [ ]:
# Find rows in 'region' column that contain '/'
rows_with_q = exchanges_2[exchanges_2['region'].str.contains('\?', na=False)]

# Display the rows
rows_with_q[['region', 'department', 'city']]

,region,department,city
76,Dubrova?ko-Neretvanska ´upanija,Op?ina Dubrovnik,Dubrovnik
940,Województwo ?ódzkie,Pabianicki,Konstantynów ?ódzki
1616,Municipiul Bucure?ti,Municipiul Bucure?ti,Bucure?ti
2460,St?edo?eský Kraj,Praha-Západ,Davle
3333,Over?ssel,Steenwijkerland,Steenwijk
...,...,...,...
11968718,Ma?opolskie,Kraków,Kraków
11969405,Over?ssel,Zwolle,Zwolle
11969552,Over?ssel,Wierden,Wierden
11969553,Województwo Dolno?l?skie,Wroc?aw,Wroc?aw


In [ ]:
# Count rows in 'region' column that contain '?'
count_rows_with_q = exchanges_2['region'].str.contains('\?', na=False).sum()

# Display the count
print(f"Number of rows in 'region' column that contain '?': {count_rows_with_q}")

Number of rows in 'region' column that contain '?': 21950


In [ ]:
# Count rows in 'department' column that contain '?'
count_rows_with_q = exchanges_2['department'].str.contains('\?', na=False).sum()

# Display the count
print(f"Number of rows in 'department' column that contain '?': {count_rows_with_q}")

Number of rows in 'department' column that contain '?': 34182


In [ ]:
# Count rows in 'city' column that contain '?'
count_rows_with_q = exchanges_2['city'].str.contains('\?', na=False).sum()

# Display the count
print(f"Number of rows in 'city' column that contain '?': {count_rows_with_q}")

Number of rows in 'city' column that contain '?': 13799


##**Drop rows that contain '?'**

*  region, department, city

In [ ]:
#drop rows that contain '?' in city, region, department

#original rows = 11969751
#rows to delete = 34182 = 0,2%

In [ ]:
exchanges_2['region'].str.contains('\?', na=False).sum()

21950

In [ ]:
exchanges_2['department'].str.contains('\?', na=False).sum()

34182

In [ ]:
exchanges_2['city'].str.contains('\?', na=False).sum()

13799

In [ ]:
exchanges_2.shape

(10061939, 23)

In [ ]:
# Find the rows to delete where 'region' matches the pattern
to_delete_region_q = exchanges_2['region'].str.contains('\?', na=False)

In [ ]:
to_delete_region_q.sum()

21950

In [ ]:
exchanges_3 = exchanges_2[~to_delete_region_q]

In [ ]:
# Find the rows to delete where 'department' matches the pattern
to_delete_dep_q = exchanges_2['department'].str.contains('\?', na=False)

In [ ]:
exchanges_3 = exchanges_2[~to_delete_dep_q]

In [ ]:
to_delete_dep_q.sum()

34182

In [ ]:
# Find the rows to delete where 'city' matches the pattern
to_delete_city_q = exchanges_2['city'].str.contains('\?', na=False)

In [ ]:
to_delete_city_q.sum()

13799

In [ ]:
exchanges_3 = exchanges_2[~to_delete_city_q]

# **Version: exchanges_3**

*  Version with no rows that contain '?'

In [ ]:
exchanges_3.shape

#original = (10062947, 23)
#version_2 = (10061939, 23)
#version_3 = (10048140, 23)

#total deleted rows = 14,807 ~ 0,15%

(10048140, 23)

In [ ]:
exchanges_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10048140 entries, 0 to 11969750
Data columns (total 23 columns):
 #   Column                Dtype              
---  ------                -----              
 0   conversation_id       Int64              
 1   exchange_id           Int64              
 2   created_at            datetime64[ns]     
 3   creator_id            Int64              
 4   guest_user_id         Int64              
 5   host_user_id          Int64              
 6   finalized_at          datetime64[us, UTC]
 7   canceled_at           datetime64[us, UTC]
 8   start_on              datetime64[ns]     
 9   end_on                datetime64[ns]     
 10  guest_count           Int64              
 11  night_count           Int64              
 12  user_cancellation_id  Int64              
 13  exchange_type         object             
 14  home_type             object             
 15  residence_type        object             
 16  capacity              Int64            

#**Load Version into Google BigQuery**

In [ ]:
import pandas_gbq

In [ ]:
pandas_gbq.to_gbq(exchanges_3, 'home_exchange.exchanges_3', project_id='savvy-torch-421309')

100%|██████████| 1/1 [00:00<00:00, 132.73it/s]


##Check datatypes in Google Bigquery

* ➰ noted that all datetime fields added with UTC timestamp when not needed